In [1]:
# bibliotecas
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader, ConcatDataset
import numpy as np
import kagglehub
from collections import Counter
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, Subset
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold

## Escolha da Tarefa de Aprendizado

A tarefa de aprendizado de máquina definida neste trabalho é a classificação de imagens binária, com o objetivo de identificar se uma imagem é real (fotografia de pessoas reais) ou sintética, ou seja, gerada por Inteligência Artificial. Como se trata de um problema visual e a tarefa envolve reconhecer padrões sutis na textura, iluminação e traços que podem diferenciar uma imagem real de uma sintética, optamos por utilizar uma Rede Neural Convolucional (CNN), que é uma arquitetura especialmente eficaz para extração automática de características em tarefas de classificação de imagens.

### Leitura do dataset

In [2]:
path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")

In [ ]:
train_path = f"{path}/cifake/train"
test_path = f"{path}/cifake/test"

### Pré-processamento

In [ ]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder(root="../data/train", transform=transform)
test_data = datasets.ImageFolder(root="../data/test", transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

### Divisão Holdout

In [ ]:
train_labels = train_data.targets
train_label_counts = Counter(train_labels)
train_label_counts # 0: FAKE / 1: REAL

Counter({0: 50000, 1: 50000})

In [13]:
train_data.classes

['FAKE', 'REAL']

O conjunto de dados de treino já está balanceado, contendo 50.000 imagens reais (classe 1) e 50.000 imagens geradas por IA (classe 0). Isso significa que o modelo será treinado com a mesma quantidade de exemplos para cada classe, o que ajuda a evitar viés de aprendizado. Com esse equilíbrio, a CNN tem maiores chances de aprender a distinguir padrões relevantes de cada classe de forma justa, sem favorecer uma categoria em detrimento da outra, o que pode resultar em melhor desempenho e métricas mais confiáveis.


Neste trabalho, optamos por utilizar a validação do tipo holdout estratificado, dividindo o conjunto de dados em 70% para treino, 15% para validação e 15% para teste. Embora a validação cruzada k-fold ofereça uma avaliação mais robusta por repetir o treinamento múltiplas vezes com diferentes divisões dos dados, ela também apresenta um custo computacional significativamente maior, especialmente quando se utiliza CNNs e um volume elevado de dados, como no caso deste projeto. O holdout, por sua vez, permite uma separação mais simples e rápida, com desempenho adequado quando há uma boa quantidade de dados e as classes estão balanceadas. Por isso, a escolha pelo holdout foi feita visando um equilíbrio entre qualidade de avaliação e viabilidade computacional.

In [14]:
train_labels = train_data.targets

In [15]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
train_idx, val_idx = next(split.split(np.zeros(len(train_labels)), train_labels))

In [16]:
train_split = Subset(train_data, train_idx)
val_split = Subset(train_data, val_idx)

In [17]:
print(f"Treino: {len(train_split)} imagens")
print(f"Validação: {len(val_split)} imagens")
print(f"Teste: {len(test_data)} imagens")

Treino: 70000 imagens
Validação: 30000 imagens
Teste: 20000 imagens
